In [ ]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
plt.style.use('seaborn-muted') # unbunter Stil
import numpy as np 
import re

In [ ]:
df = pd.read_csv('SIK_ISEA_Personen.csv', sep = ";")

Informationen zu Umfang und Inhalt des Datensatzes

In [ ]:
df.dtypes # Ermitteln der Datentypen

In [ ]:
df.columns # Spaltennamen mithilfe des columns-Attributes ermitteln

In [ ]:
df.head(2) # erste Zeilen ausgeben mit df.head(n)

In [ ]:
df.tail(2) # letzte Zeilen ausgeben

In [ ]:
df.count(axis=0, numeric_only=False)

# Anzahl Werte pro Spalte zählen, die vorhanden, d.h. nicht-NA sind. 
# Mit dem Axis-Wert 0 oder 'index' werden die Werte pro Spalte gezählt.
# Mit  axis=1 oder ‘columns’ würden die Werte jeder Zeile gezählt

In [ ]:
# erste Visualisierung mit Pandas - Säulendiagramm
# Die ursprüngliche Grafik war etwas klein und wurde vergrössert

df.count(axis=0, numeric_only=False).plot(figsize=(10,6),kind='bar',
                                  color='forestgreen', title="ANZAHL WERTE DER EINZELNEN SPALTEN") 

In [ ]:
# Spalten, die für die Analyse nicht benötigt werden, werden entfernt
df = df.drop(['NUTZUNGSLIZENZ', 'GND', 'HLS_ID', 'SIKART_LINK'], axis=1)
df.head(3)

Spalten "Bereich" und "Typus" auf ihre Begriffe und Häufigkeit anschauen

In [ ]:
df['BEREICH'].unique()  # Jeden Begriff der Spalte Bereich ausgeben. In der Liste wurden einzelne Personen, Familien und Gruppen erfasst

In [ ]:
# BEREICH wird als Key ausgewählt, HAUPTNR als Index
bereich = df.groupby("BEREICH")['HAUPTNR'].count()
bereich.plot(figsize=(8,3), kind='barh', color='orchid', title="VERTEILUNG DER SPALTE BEREICH")
plt.ylabel('') 

# Entfernung des Spaltennamens bzw. der Beschriftung der y-Achse
# Alternative Schreibweise: df.groupby("BEREICH")['HAUPTNR'].count().plot(kind='barh', color='orchid', title="...") 
# Balkendiagramm: mit barh = bar horizontal wird das Diagramm gedreht
# Beim überwiegenden Teil der Instanzen im Datensatz handelt es sich um Personen, Gruppen und Familien sind vernachlässigbar

In [ ]:
df['TYPUS'].unique() # Begriffe der Spalte "Typus" ausgeben. Es handelt sich um Berufe

In [ ]:
df.groupby("TYPUS")['HAUPTNR'].count() # Häufigkeit der charakteristischen Berufe ermitteln

In [ ]:
# Die Unterscheidung, ob der Begriff FotografIn aufgrund eines Lexikon-Eintrages vergeben wurde oder nicht, spielt hier keine Rolle
# Der Begriff (Lexikon) wird mit einer regular expression entfernt
# Klammern () müssen in eckigen Klammern [] geschrieben werden, damit sie ersetzt werden
df['TYPUS'] = df['TYPUS'].replace(to_replace={' [(]Lexikon[)]': ''}, regex=True)

In [ ]:
berufe = df.groupby("TYPUS")['HAUPTNR'].count() # Häufigkeit der charakteristischen Berufe wird neu gezählt
print(berufe)

In [ ]:
berufe.plot(figsize=(8,4), kind='barh', color='slateblue', title="BERUFE") 
plt.ylabel('')

Häufigkeit der Namen

In [ ]:
name = df.groupby('NAME')['HAUPTNR'].count().sort_values(ascending=False) 
print(name)

In [ ]:
np.sum(name > 10) 
# Mit einem booleschen Array können die Anzahl Einträge, bzw. Namen gezählt werden, die mehr als x mal auftreten (JVDPL S.92)
# 126 Namen treten mehr als 10 mal im Datensatz auf

In [ ]:
np.sum(name == 1) 
# 6472 Namen treten genau einmal im Datensatz auf

In [ ]:
np.sum((name > 1) & (name <= 10)) 
# Mithilfe der bitweisen Logikoperatoren von Python lässt sich feststellen, wie viele Namen 2 bis 10 mal auftreten (JVDPL S. 93)

In [ ]:
topname = name[name > 25]# Namen, die mehr als 25mal auftreten mittels Maskierung ermitteln (JVDPL S.94)
print(topname)

In [ ]:
topname.plot(figsize=(9, 9), kind='pie', title='HÄUFIGSTE NAMEN',cmap='ocean')
plt.ylabel('') 

In [ ]:
 # damit die weiblichen Vornamen ausgewählt werden können, muss die Spalte "TYPUS" als Index gesetzt werden
df_typus = df.set_index("TYPUS")
# Auswahl der Zeilen "Künstlerin", sowie der Spalten Vorname und Hauptnr.
# Auswahl von 2 Spalten, damit ein Dataframe bestehen bleibt und ein Pie-Plot möglich ist
df_typus_weiblich = df_typus.loc[["Künstlerin"], ['VORNAME', 'HAUPTNR']] 
wvorname = df_typus_weiblich.groupby('VORNAME')['HAUPTNR'].count().sort_values(ascending=False)
print(wvorname)

In [ ]:
wvorname_top = wvorname[wvorname > 15]
wvorname_top.plot(figsize=(10,10), kind='pie',x='VORNAME',y='HAUPTNR', title='HÄUFIGSTE WEIBLICHE VORNAMEN', cmap='Spectral')
plt.ylabel('')

Analyse und Plot-Variationen Geburtsländer

In [ ]:
df['GEBURTSLAND'].unique() # Ausgabe sämtlicher Begriffe. Diese bestehen meistens nur aus den Länderkürzeln

In [ ]:
df['GEBURTSLAND'].nunique() # Ausgabe der Anzahl Länder

In [ ]:
land = df.groupby("GEBURTSLAND")['HAUPTNR'].count().sort_values(ascending=False)
print(land)

In [ ]:
# Ausgabe mit einfachem Pandas-Plot
# Art des Diagrammes als Methode .line und nicht als Keyword-Argument
# Die Beschriftung der x-Achse "GEBURTSLAND" konnte nicht entfernt werden

land.plot.line(figsize=(8,4), color='green', title='VERTEILUNG DER GEBURTSLÄNDER') 

In [ ]:
# Besser ist die Ausgabe mit einem Streudiagramm, da es sich bei Ländern um distinkte Einheiten handelt
plt.figure(figsize=(18,4))
ax = plt.axes()
plt.plot(land, 'o', color='red')
ax.set(xlim=(-1, 120), ylim=(-1000, 14000), title='VERTEILUNG DER GEBURTSLÄNDER')
plt.xlabel('xlabel', fontsize=1)
plt.xticks(rotation=90);

In [ ]:
 # Logarithmische Darstellung der Y-Achse

plt.figure(figsize=(18,4))
ax = plt.axes()
plt.plot(land, 'o', color='red')
plt.yscale('log')
ax.set(xlim=(-1, 120), title='VERTEILUNG DER GEBURTSLÄNDER')
plt.xlabel('xlabel', fontsize=1)
plt.xticks(rotation=90);

In [ ]:
# Obwohl Familien und Gruppen kaum vorkommen, sollen  sie für die weitere Auswertung nach Geburtsländern nicht berücksichtigt werden

df_bereich = df.set_index("BEREICH") # Spalte "BEREICH" in Index umfunktionieren
df_person = df_bereich.loc["Person"]# Nur Zeilen mit dem Index-Wert Person ausgeben

In [ ]:
pers_land = df_person.GEBURTSLAND.value_counts()

# Gibt eine Series aus in welchen Ländern die meisten Personen geboren sind, Sortiert automatisch, NA-Werte werden gelöscht. 
print(pers_land)

In [ ]:
# Länder, die weniger als 1o mal auftreten, unter "Andere" zusammengefasst
threshold = 10
mask = pers_land > threshold
tail_pers_land = pers_land.loc[~mask].sum()
pers_land = pers_land.loc[mask]

In [ ]:
# Werte werden neu sortiert, damit die Ländergruppe "Andere" nicht am Schluss angezeigt wird
print(pers_land)
pers_land['ANDERE'] = tail_pers_land
pers_land = pers_land.sort_values(ascending=False) 

In [ ]:
 # Logarithmische Darstellung der Y-Achse für Länder > 10 Werte
plt.figure(figsize=(14,4))
ax = plt.axes()
plt.plot(pers_land, 'o', color='red')
plt.yscale('log')
ax.set(title='VERTEILUNG DER GEBURTSLÄNDER')
plt.xticks(rotation=90);

In [ ]:
# Geburtsländer ohne Schweiz
ohne_ch = pers_land.drop(labels=['CH'])
print(ohne_ch)

In [ ]:
fig, ax = plt.subplots(figsize=(18, 5))
sns.barplot(x=ohne_ch.index, y=ohne_ch) 
ax.set(title='GEBURTSLÄNDER OHNE SCHWEIZ', ylabel='')

Geburtsjahre - Sterbejahre - Lebensdauer

In [ ]:
geburtsjahr = pd.Series(df['GEBURTSJAHR']).dropna()
geburtsjahr = geburtsjahr.astype('int').sort_values(ascending=False)
print(geburtsjahr)

In [ ]:
geburtsjahr.plot(kind='hist', figsize=(4,5), title='GEBURTSJAHRE', color='yellowgreen')
plt.ylabel('')

In [ ]:
sterbejahr = pd.Series(df['STERBEJAHR']).dropna()
sterbejahr = sterbejahr.astype('int').sort_values(ascending=False)
print(sterbejahr)

In [ ]:
sterbejahr.plot(kind='hist', figsize=(4,5), title='STERBEJAHRE')
plt.ylabel('')

In [ ]:
fig, ax = plt.subplots()
geburtsjahr.plot(kind='hist', color='yellowgreen')
sterbejahr.plot(kind='hist', figsize=(4,5), title='GEBURTS- UND STERBEJAHRE')
plt.ylabel('')

In [ ]:
# Density-Plot als Spezialfall eines Histogramms
fig, ax = plt.subplots()
df['GEBURTSJAHR'].plot.density(figsize=(6,4), title='GEBURTS- UND STERBEJAHRE', color='yellowgreen', linewidth=(6))
df['STERBEJAHR'].plot.density(linewidth=(6))
plt.ylabel('')

In [ ]:
# Subplots mit objektorientierter Programmierung

fig = plt.figure(figsize=(10,8))
sub1 = fig.add_subplot(221)
sub1.set_title('GEBURTSJAHRE')
geburtsjahr.plot(kind='hist', color='yellowgreen')
plt.ylabel('')

sub2 = fig.add_subplot(222)
sub2.set_title('STERBEJAHRE')
sterbejahr.plot(kind='hist')
plt.ylabel('')

In [ ]:
with sns.axes_style('white'):
    g = sns.catplot("GEBURTSJAHR",data=df, aspect=4, height=3, kind="strip", color='red')
    g = sns.catplot("STERBEJAHR", data=df, aspect=4, height=3, kind="strip", color='steelblue')
    g.set_xticklabels(step=2)

Lebensdauer 

In [ ]:
# Bei einem DataFrame kann die gleiche Syntax wie bei einem Dictionary verwendet werden,
# d.h. Spalten können z.B. subtrahiert werden

df['LEBENSDAUER'] = df['STERBEJAHR'] - df['GEBURTSJAHR']
lebensdauer = pd.Series(df['LEBENSDAUER']).dropna()
lebensdauer = lebensdauer.astype('int').sort_values(ascending=False)
print(lebensdauer)

# es werden unrealistische Lebensjahre ausgegeben

In [ ]:
# Variante zu oben. Es werden zunächst die NAN-Werte von 2 bestimmten Spalten gelöscht, d.h. die Zeilen mit Nan.Werten der jeweiligen Spalten
df2 = df.dropna(subset=['GEBURTSJAHR', 'STERBEJAHR'])
Lebensdauer = df2['STERBEJAHR'] - df2['GEBURTSJAHR']
Lebensdauer = Lebensdauer.astype('int').sort_values(ascending=False)
print(Lebensdauer)

In [ ]:
# Zeilen prüfen, die eine unrealistische Lebensdauer ergeben
df.loc[10012] # Es handelt sich um eine Künstler-Familie

In [ ]:
df.loc[12330] 
# Die Lebensdaten sind nicht bekannt.
# erfasst wurden Jahre, in denen die Person nachweislich Kunstwerke erschaffen hat (siehe SIKART)

In [ ]:
# Test mit Masking
lebenkurz = lebensdauer[lebensdauer > 18] 
lebenlang = lebenkurz[lebenkurz < 108]
print(lebenlang)

In [ ]:
# Zumindest Familien und Gruppen können gelöscht werden, 
# Zuerst Spalte Bereich als Index definieren
# Person als Index und alle Spalten auswählen

df_bereich = df.set_index("BEREICH") 
df_person = df_bereich.loc[["Person"], :] 
df_person['LEBENSDAUER'] = df_person['STERBEJAHR'] - df_person['GEBURTSJAHR']

In [ ]:
df_person.columns # prüfen, ob die Spalte Lebensdauer ergänzt wurde

In [ ]:
leben = pd.Series(df_person['LEBENSDAUER']).dropna()
leben = leben.astype(int)
print(leben)

In [ ]:
# hier gelang es nicht, einen Plot auszugeben
# leben.plot(kind='bar')

In [ ]:
# Variante mit Seaborn
# zuerst müssen die Nan-Werte entfernt werden

df = df.dropna(subset=['LEBENSDAUER'])
df['LEBENSDAUER'] = df['LEBENSDAUER'].astype(int)

In [ ]:
with sns.axes_style('white'):
    g = sns.catplot("LEBENSDAUER", data=df, aspect=2.5, kind="count")
    (g.set_xticklabels(step=5)
     .set_axis_labels("Lebensdauer", "Anzahl"))
    
# ax.set_title('GEBURTSJAHRE IN JAHRZEHNTEN',fontsize=5) 
# Der Titel konnte nicht eingefügt werden
# die unrealistischen Lebensjahre sind im Plot enthalten

In [ ]:
# Spalte Geburtsjahre nach Jahrzehnt wurde hinzugefügt
# Ziel war es, eine geringere Zahl Jahre zu erhalten
df['JAHRZEHNT'] = 10 * (df['GEBURTSJAHR'] // 10) 
df.columns

In [ ]:
df.head(2)

In [ ]:
df['JAHRZEHNT'].dtypes 

In [ ]:
df['JAHRZEHNT'].unique() 

In [ ]:
# Filtern der Nan-Werte und Umwandlung float zu integer 

df = df.dropna(subset=['JAHRZEHNT'])
df['JAHRZEHNT'] = df['JAHRZEHNT'].astype(int)

In [ ]:
with sns.axes_style('white'):
    g = sns.catplot("JAHRZEHNT", data=df, aspect=2.5, kind="count")
    (g.set_xticklabels(step=5)
     .set_axis_labels("Jahrzehnte", "Anzahl"))
# ax.set_title('GEBURTSJAHRE IN JAHRZEHNTEN',fontsize=5) 
# Der Titel konnte nicht eingefügt werden

Bearbeitungstiefe - Qualität der biographischen Daten

In [ ]:
df.groupby('BEARBEITUNGSTIEFE').count() # Die Bearbeitungstiefe wird mit Sternchen angegeben

In [ ]:
# Die Sternchen in der Spalte Bearbeitungstiefe sollen durch numerische Werte (Integers) ersetzt werden
df['BEARBEITUNGSTIEFE'] = df['BEARBEITUNGSTIEFE'].dropna().replace("*****", int(5)).replace("****", int(4)).replace("***", int(3)).replace("**", int(2)).replace("*", int(1))


In [ ]:
sns.catplot(
    data=df,
    x='BEARBEITUNGSTIEFE',
    y='HAUPTNR',
    kind='strip',
    height=5, 
    aspect=3) # Breite

In [ ]:
sns.catplot(
    data=df,
    x='BEARBEITUNGSTIEFE',
    kind='count',
    height=5, # make the plot 5 units high
    aspect=3) # height should be three times width

In [ ]:
# Die Bearbeitungstiefe der SIKART-Daten wird nicht durch das Geburtsjahr bestimmt
sns.relplot(y='BEARBEITUNGSTIEFE', x='GEBURTSJAHR',
            sizes=(40, 400), alpha=.5
            , palette="muted",
            height=6, data=df)